In [ ]:
import sys
sys.path.insert(0,'c:/MyDocs/integrated/') # adjust to your setup

%run "catalog_support.py" 

today = datetime.datetime.today()
edate = datetime.datetime(year=2024,month=3,day=5)
Numdays = (today - edate).days

showHeader('Raw Disclosures',line2=f'{Numdays} days of FracFocus changes',use_remote=True)

In [ ]:
import datetime

last_repo = datetime.datetime(year=2025,month=8,day=7)

# edate = today - datetime.timedelta(days=Numdays)
# print('earlist shown',edate)

daily_status_folder = r"G:\My Drive\webshare\daily_status"
added_disc_fn = os.path.join(daily_status_folder,'added_disc_df.parquet')
removed_disc_fn = os.path.join(daily_status_folder,'removed_disc_df.parquet')
modified_disc_fn = os.path.join(daily_status_folder,'modified_disc_df.parquet')


In [ ]:
import os
import pickle
import openFF.common.text_handlers as th
# arc_dir = r"D:\openFF_archive\diff_dicts"
#!
# arc_dir = r"C:\MyDocs\integrated\openFF_archive\diff_dicts"
arc_dir = r"G:\My Drive\production\openFF_archive\diff_dicts"

diff_fns = os.listdir(arc_dir)
download_dates = []
added = []
changed = []
removed = []
casing = set()
operator = set()


for fn in diff_fns:
    tdate = datetime.datetime(int(fn[10:14]),int(fn[15:17]),int(fn[18:20]))
    # print(tdate)
    if tdate>= edate:
        download_dates.append(fn[10:20])
        with open(os.path.join(arc_dir,fn),'rb') as f:
            diff_dic = pickle.load(f)
        if len(diff_dic['removed_disc'])>0:
            t = diff_dic['removed_disc'].copy()
            # print(t.head())
            t['date_changed'] = tdate
            t['change_type'] = 'removed'
            removed.append(t)
        if len(diff_dic['added_disc'])>0:
            t = diff_dic['added_disc'].copy()
            t['date_changed'] = tdate
            t['change_type'] = 'added'
            added.append(t)
        if len(diff_dic['changed_disc'])>0:
            t = diff_dic['changed_disc'].copy()
            t['date_changed'] = tdate
            t['change_type'] = 'modified'            
            changed.append(t)
        if len(diff_dic['casing'])>0:
            for item in diff_dic['casing']:
                # print(item)
                if item[1]==None: ig = ''
                else: ig = item[1].strip().lower()
                tup = (item[0],ig)
                casing.add(tup)
        if len(diff_dic['OperatorName'])>0:
            for item in diff_dic['OperatorName']:
                operator.add(item)

alllists = added + changed + removed
wholeset = pd.concat(alllists,sort=True)
# print(f'Len wholeset: {len(wholeset)}')
# wholeset

In [ ]:
gb_added = wholeset[wholeset.change_type=='added'].groupby('date_changed').size()
added_sum = gb_added.resample("D").sum()
added_sum = added_sum+.001 # to distinguish between zeros and no data

gb_removed = wholeset[wholeset.change_type=='removed'].groupby('date_changed').size()
removed_sum = gb_removed.resample("D").sum()
removed_sum = removed_sum+0.001

gb_changed = wholeset[wholeset.change_type=='modified'].groupby('date_changed').size()
changed_sum = gb_changed.resample("D").sum()
changed_sum = changed_sum+0.001

In [ ]:
#### set up the difflib outputs

import difflib
archive_folder = r"G:\My Drive\production\openFF_archive\raw_dataframes"  #drive
# archive_folder = r"C:\MyDocs\integrated\openFF_archive\raw_dataframes" # production machine

def get_raw_list():
    alst = os.listdir(archive_folder)
    dates = []
    for f in alst:
        dates.append(f[7:17])
    return dates
    
def get_early_late_fn(change_date,datelst):
    latefn = os.path.join(archive_folder,'raw_df_'+change_date+'.parquet')
    idx = datelst.index(change_date)
    earlyfn = os.path.join(archive_folder,'raw_df_'+datelst[idx-1]+'.parquet')
    # print(earlyfn+'\n'+latefn)
    return earlyfn,latefn

def get_comparison_dfs(apinum,eDId,lDId,earlyfn,latefn):
    # Have to fetch by discID otherwise will pickup multiple disclosures
    filt = [('DisclosureId','==',eDId)]
    edf = pd.read_parquet(earlyfn,filters=filt)
    filt = [('DisclosureId','==',lDId)]
    ldf = pd.read_parquet(latefn,filters=filt)
    return edf,ldf
    

In [ ]:
import difflib

def get_local_diff_fn(api,change_date):
    # return os.path.join(r"C:\Users\Gary\My Drive\webshare\daily_status\diff_files",api+'_'+change_date+'.html')
    return os.path.join(r"G:\My Drive\webshare\daily_status\diff_files",api+'_'+change_date+'.html')

def get_diff_url(api,change_date):
    return "https://storage.googleapis.com/open-ff-browser/difflib/"+api+'_'+change_date+'.html'

def make_logistics_summary(ddf):
    # assumes single disclosure in ddf
    # start with meta or logistics data, so all is available in any line
    jsd = ddf.JobStartDate.iloc[0]
    jed = ddf.JobEndDate.iloc[0]
    state = ddf.StateName.iloc[0]
    county = ddf.CountyName.iloc[0]
    api = ddf.APINumber.iloc[0]
    operator = ddf.OperatorName.iloc[0]
    wellname = ddf.WellName.iloc[0]
    lat = ddf.Latitude.iloc[0]
    lon = ddf.Longitude.iloc[0]
    datum = ddf.Projection.iloc[0]
    fed = ddf.FederalWell.iloc[0]
    indian = ddf.IndianWell.iloc[0]
    tvd = ddf.TVD.iloc[0]
    tbwv = ddf.TotalBaseWaterVolume.iloc[0]
    tbnwv = ddf.TotalBaseNonWaterVolume.iloc[0]
    did = ddf.DisclosureId.iloc[0]
    ffv = ddf.FFVersion.iloc[0]

    s=  '\n**Logistics Section** \n'
    s+= f'APINumber: {api}\n'
    s+= f'Job Start Date: {jsd}\n'
    s+= f'Job End Date: {jed}\n'
    s+= f'State: {state}\n'
    s+= f'County: {county}\n'
    s+= f'Operator: {operator}\n'
    s+= f'Well Name: {wellname}\n'
    s+= f'Latitude: {lat}  Longitude: {lon}\n'
    s+= f'Projection: {datum}\n'
    s+= f'Federal: {fed}   Indian: {indian}\n'
    s+= f'Total Vertical Depth: {tvd}\n'
    s+= f'Carrier water volume: {tbwv}\n'
    s+= f'Vol of non-water carrier: {tbnwv}\n'
    s+= f'FF Version: {ffv}\n'
    s+= f'DisclosureId: {did}\n'

    return s

def make_products_summary(ddf):
    ###  TradeName products
    ddf.CASNumber = ddf.CASNumber.fillna('--empty--')
    ddf = ddf.sort_values(['TradeName','CASNumber'])
    gb = ddf.groupby('TradeName')['CASNumber'].apply(set).reset_index()
    s= '\n**Products List Section with components**\n'
    for i,row in gb.iterrows():
        s+= f'Name: {row.TradeName}\n'
        lst = list(row.CASNumber)
        lst.sort()
        for cas in lst:
            s+= f'      - {cas}\n'
    return s
    # tn = ddf.TradeName.unique().tolist()
    # tn.sort()
    # for i in tn:
    #     s+= f'Name: {i}\n'

def make_record_count_summary(ddf):
    ###  Number of records
    cn = ddf.CASNumber.notna()&ddf.PercentHFJob.notna()
    s= '\n**Number of records Section**\n'
    s+= f'Total num records: {len(ddf)}\n'
    s+= f'Record with chem info: {len(ddf[cn])}\n'
    return s

def make_num_duplicates(ddf):
    cn = ddf.CASNumber.notna()&ddf.IngredientName.notna()&ddf.PercentHighAdditive.notna()&ddf.PercentHFJob.notna()
    cdup = ddf[['CASNumber','IngredientName','PercentHighAdditive','PercentHFJob','MassIngredient']].duplicated()
    s = '\n**Number of duplicate records**\n'
    t = ddf[cn&cdup].copy()
    s += f'Number of duplicate records: {len(t)}\n'
    # for i,row in t.iterrows():
    #     s += f'duplicated record: {row.CASNumber}, {row.IngredientName}, {row.PercentHFJob}\n'
    return s

def make_chem_list(ddf):
    #  List of UNIQUE CAS/ING
    s = '\n**Unique CASNumber/IngredientName pairs**\n'
    t = ddf.groupby(['CASNumber','IngredientName'],as_index=False).size().rename({'size':'num_recs'},axis=1)
    t = t.sort_values(['CASNumber','IngredientName'])
    for i,row in t.iterrows():
        s += f'CAS: {row.CASNumber}, Name: {row.IngredientName}, How many: {row.num_recs}\n'
    return s

def make_chem_data_list(ddf):
    # list of main components of a data record - only those with CASNumber or IngredientName
    s = '\n**record data**\n\n'
    sortby_order = ['TradeName','Purpose','Supplier','PercentHFJob','CASNumber','IngredientName']
    # t = ddf.groupby(['CASNumber','IngredientName'],as_index=False).size().rename({'size':'num_recs'},axis=1)
    t = ddf.sort_values(sortby_order)
    for i,row in t.iterrows():
        s += f'{row.TradeName};{row.Purpose};{row.Supplier}\n\t{row.CASNumber}\n\t{row.IngredientName}\n\t{row.PercentHighAdditive};{row.PercentHFJob};{row.MassIngredient}\n'
    return s
    
# def make_diff_str(str1,str2,section='logistics'):
#     diff = difflib.context_diff(str1, str2,n=20,fromfile='', tofile='')
#     s = f'\n\n<<{section}>>\n'
#     for line in diff:
#         s+= line
#     # print(f'*****   DIFF FILE FOLLOWS *****\n{s}')
#     return s

def make_html_diff(str1,str2):
    HD = difflib.HtmlDiff(wrapcolumn=80)
    return HD.make_file(str1, str2,fromdesc='original', todesc='modified')

def get_diff_size(str1,str2):
    diff = difflib.context_diff(str1, str2,fromfile='original', tofile='modified')
    s = ''
    for line in diff:
        s+= line
    return len(s)

# def comparison_alerts(edf,ldf):
    
    
def PDF_like_compare(apinumber,eDId,lDId,change_date,datelst,outhtml):
    print(f'working on {apinumber} {change_date}')
    earlyfn,latefn = get_early_late_fn(change_date=change_date,datelst=datelst)
    edf,ldf = get_comparison_dfs(apinumber,eDId,lDId,earlyfn,latefn)
    try:
        str1 = f'Archive file :\n\t{earlyfn}\n'
        str1 += make_logistics_summary(edf)
        str1 += make_products_summary(edf)
        str1 += make_record_count_summary(edf)
        str1 += make_num_duplicates(edf)
        str1 += make_chem_list(edf)
        str1 += make_chem_data_list(edf)
        lst1 = str1.split('\n')
        str2 = f'Archive file :\n\t{latefn}\n'
        str2 += make_logistics_summary(ldf)
        str2 += make_products_summary(ldf)
        str2 += make_record_count_summary(ldf)
        str2 += make_num_duplicates(ldf)
        str2 += make_chem_list(ldf)
        str2 +=make_chem_data_list(ldf)
        lst2 = str2.split('\n')
        # print(str1)
        out  = make_html_diff(lst1,lst2)
        with open(outhtml,'w') as f:
            f.write(out)
        return(get_diff_size(lst1,lst2))
    except:
        print('exception in PDF_like_comparison')
        return -1


## Summary of ECMC disclosure list

In [ ]:
# get current ECMC disclosures and add to table
url = "https://ecmc.state.co.us/depot/Stats/ChemicalDisclosures/Index/Locations"
tables = pd.read_html(url)
t = tables[0]
t

In [ ]:
## This cell is gemini code to archive changed location pages at ECMC and create difflib logs of changes.

import requests
from bs4 import BeautifulSoup
import os
from pathlib import Path
from datetime import datetime
import logging
from urllib.parse import urljoin, urlparse, urlunparse
import sys
import difflib

# --- Configuration ---
INDEX_URL = "https://ecmc.state.co.us/depot/Stats/ChemicalDisclosures/Index/Locations"
ARCHIVE_DIR = Path(r"G:\My Drive\production\openFF_archive\ecmc_loc_data")
USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"

# --- Helper Functions ---

def get_page_content(url, session):
    headers = {'User-Agent': USER_AGENT}
    try:
        response = session.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        response.encoding = response.apparent_encoding or 'utf-8'
        return response.text
    except requests.exceptions.RequestException as e:
        logging.error(f"Error fetching {url}: {e}")
        return None

def extract_location_number(url):
    try:
        path = urlparse(url).path
        parts = path.strip('/').split('/')
        if len(parts) > 0 and parts[-1].isdigit():
             if len(parts) > 1 and parts[-2].lower() == 'location':
                 return parts[-1]
             else:
                 logging.warning(f"URL {url} structure might differ, but found trailing number: {parts[-1]}")
                 return parts[-1]
        logging.warning(f"Could not extract location number from URL: {url}")
        return None
    except Exception as e:
        logging.error(f"Error parsing URL {url} for location number: {e}")
        return None

def get_most_recent_archive(location_num, archive_dir):
    """
    Finds the most recent ARCHIVE file (excluding difflib files)
    for a given location number.
    """
    prefix = f"Loc{location_num}_"
    try:
        all_matching_files = list(archive_dir.glob(f"{prefix}*.html"))
        archive_files = [
            f for f in all_matching_files
            if not f.name.endswith("_difflib.html")
        ]
        if not archive_files:
            return None
        archive_files.sort()
        return archive_files[-1]
    except OSError as e:
        logging.error(f"Error accessing archive directory {archive_dir} while searching for files: {e}")
        return None

def extract_relevant_content(html_content):
    """
    Parses HTML and extracts text content from specific sections
    relevant for change detection. Normalizes whitespace.
    Returns a single string of relevant text, or None if extraction fails.
    """
    if not html_content:
        return None
    try:
        soup = BeautifulSoup(html_content, 'html.parser')
        relevant_texts = []
        data_container_divs = soup.find_all('div', class_='mx-auto p-2 w-md-75 w-100', limit=3)

        if len(data_container_divs) >= 1:
            text1 = ' '.join(data_container_divs[0].get_text(separator=' ', strip=True).split())
            relevant_texts.append(text1)
        if len(data_container_divs) >= 3:
             text3 = ' '.join(data_container_divs[2].get_text(separator=' ', strip=True).split())
             relevant_texts.append(text3)
        else:
            logging.debug("Main container structure (3 divs) not found/matched. Falling back to direct table search.")
            tables = soup.find_all('table', class_='b-table table table-striped table-sm')
            if tables:
                 for i, table in enumerate(tables):
                     table_text = ' '.join(table.get_text(separator=' ', strip=True).split())
                     if table_text not in relevant_texts:
                          relevant_texts.append(table_text)
                 logging.debug(f"Found {len(tables)} table(s) via fallback.")
            else:
                # Only warn if the primary method also failed
                if len(data_container_divs) < 1:
                     logging.warning("Could not find relevant data containers or tables.")

        if not relevant_texts:
            logging.error("Failed to extract any relevant content chunks.")
            return None

        return "\n---\n".join(relevant_texts)

    except Exception as e:
        logging.error(f"Error parsing HTML with BeautifulSoup: {e}")
        return None

# --- Main Script Logic ---
def ECMC_archive_main():
    # --- Logging Setup ---
    log_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    logger = logging.getLogger()
    logger.setLevel(logging.INFO) # Set level (INFO, DEBUG, etc.)

    try:
        ARCHIVE_DIR.mkdir(parents=True, exist_ok=True)
        print(f"Archive directory checked/created: {ARCHIVE_DIR}")
    except OSError as e:
        print(f"CRITICAL: Could not create or access archive directory {ARCHIVE_DIR}: {e}", file=sys.stderr)
        return

    log_file_path = ARCHIVE_DIR / "todays-log.txt"
    try:
        file_handler = logging.FileHandler(log_file_path, mode='w', encoding='utf-8')
        file_handler.setFormatter(log_formatter)
        logger.addHandler(file_handler)
        print(f"Logging to file: {log_file_path}")
    except OSError as e:
        print(f"ERROR: Could not create or write to log file {log_file_path}: {e}", file=sys.stderr)
        print("WARNING: Proceeding without file logging.", file=sys.stderr)

    # --- Start of Process ---
    logging.info("Starting archive process...")
    today_date_str = datetime.now().strftime("%Y-%m-%d")
    current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    new_files_saved = 0
    updated_files_saved = 0
    diff_files_saved = 0
    links_processed = 0
    links_failed = 0
    links_found_count = 0

    with requests.Session() as session:
        session.headers.update({'User-Agent': USER_AGENT})

        logging.info(f"Fetching index page: {INDEX_URL}")
        index_content = get_page_content(INDEX_URL, session)
        if not index_content:
            logging.critical(f"Failed to fetch index page. Exiting.")
            print("CRITICAL: Failed to fetch index page. Exiting.", file=sys.stderr)
            return
        soup = BeautifulSoup(index_content, 'html.parser')
        location_links = soup.find_all('a', href=lambda href: href and '/Location/' in href)
        links_found_count = len(location_links)
        logging.info(f"Found {links_found_count} potential location links.")
        LINK_BASE_URL = "https://ecmc.state.co.us/depot/"

        for link in location_links:
            links_processed += 1
            location_url = ""
            try:
                relative_url = link.get('href')
                if not relative_url:
                    logging.warning(f"Link tag found without href attribute. Skipping.")
                    links_failed += 1
                    continue

                location_url = urljoin(LINK_BASE_URL, relative_url)
                location_num = extract_location_number(location_url)
                if not location_num:
                    links_failed += 1
                    continue

                logging.info(f"Processing Location: {location_num} ({location_url})")

                target_filename = ARCHIVE_DIR / f"Loc{location_num}_{today_date_str}.html"
                most_recent_file = get_most_recent_archive(location_num, ARCHIVE_DIR)

                current_content_full = get_page_content(location_url, session)
                if current_content_full is None:
                    logging.warning(f"Failed to fetch content for location {location_num}. Skipping.")
                    links_failed += 1
                    continue

                current_relevant_content = extract_relevant_content(current_content_full)
                if current_relevant_content is None:
                    logging.warning(f"Failed to extract relevant content from current page for {location_num}. Skipping comparison.")
                    links_failed += 1
                    continue

                save_needed = False
                is_new = False
                archived_content_full = None
                archived_relevant_content = None

                if most_recent_file:
                    logging.info(f"Checking against most recent archive: {most_recent_file.name}")
                    try:
                        archived_content_full = most_recent_file.read_text(encoding='utf-8')
                        archived_relevant_content = extract_relevant_content(archived_content_full)

                        if archived_relevant_content is None:
                            logging.warning(f"Failed to extract relevant content from archived file {most_recent_file.name}. Treating as changed.")
                            save_needed = True
                        elif current_relevant_content != archived_relevant_content:
                            logging.info(f"RELEVANT content changed for Location {location_num}. Archiving update.")
                            save_needed = True
                        else:
                            logging.info(f"Relevant content unchanged for Location {location_num}. No archive needed today.")

                    except IOError as e:
                        logging.error(f"Error reading archive file {most_recent_file} for comparison: {e}")
                        logging.info(f"Marking Location {location_num} for update due to error reading previous archive.")
                        save_needed = True

                    # --- Generate Diff using RELEVANT content if change detected ---
                    # Check we have relevant content from archive AND save is needed based on relevant comparison
                    if save_needed and archived_relevant_content is not None:
                         try:
                             logging.info(f"Generating diff file from RELEVANT content for Location {location_num}...")
                             diff_filename = ARCHIVE_DIR / f"{target_filename.stem}_difflib.html"

                             # *** Use RELEVANT content strings, split into lines ***
                             old_lines = archived_relevant_content.splitlines()
                             new_lines = current_relevant_content.splitlines()
                             # *****************************************************

                             differ = difflib.HtmlDiff(tabsize=4, wrapcolumn=80)
                             # Generate the HTML diff using the relevant lines
                             diff_html = differ.make_file(
                                 old_lines, new_lines,
                                 # Update labels to reflect content source
                                 fromdesc=f"Archived Relevant Section: {most_recent_file.name}",
                                 todesc=f"Current Relevant Section: {location_num} ({current_timestamp})",
                                 context=True, numlines=5 # Show context lines
                             )
                             # Save the generated HTML diff
                             diff_filename.write_text(diff_html, encoding='utf-8')
                             logging.info(f"Successfully saved diff of relevant sections: {diff_filename.name}")
                             diff_files_saved += 1
                         except Exception as e:
                             # Log error if diff generation fails
                             logging.error(f"Could not generate or save diff file from relevant content for {target_filename.stem}: {e}")
                    # --- End Diff Generation ---

                else: # No most_recent_file exists
                    logging.info(f"No previous archive found for Location {location_num}. Archiving.")
                    save_needed = True
                    is_new = True

                # Save the main FULL archive file if needed
                if save_needed:
                    try:
                        target_filename.write_text(current_content_full, encoding='utf-8')
                        logging.info(f"Successfully saved FULL content: {target_filename.name}")
                        if is_new:
                            new_files_saved += 1
                        else:
                            updated_files_saved += 1
                    except IOError as e:
                        logging.error(f"Failed to write file {target_filename}: {e}")
                        links_failed += 1
                    except Exception as e:
                         logging.error(f"Unexpected error writing file {target_filename}: {e}")
                         links_failed += 1

            except Exception as e:
                logging.error(f"An unexpected error occurred processing link {location_url or link.get('href', 'N/A')}: {e}")
                links_failed += 1

    # --- Log Summary to File ---
    logging.info("--- Archive Process Summary ---")
    logging.info(f"Date: {today_date_str}")
    logging.info(f"Total links found matching pattern: {links_found_count}")
    logging.info(f"Links processed attempt: {links_processed}")
    logging.info(f"New archives saved: {new_files_saved}")
    logging.info(f"Updated archives saved: {updated_files_saved}")
    logging.info(f"Diff files saved: {diff_files_saved}")
    logging.info(f"Links skipped/failed: {links_failed}")
    logging.info("Archive process finished.")

    # --- Print Summary to Standard Output ---
    print("\n--- ECMC Archive Process Summary ---")
    # print(f"Date: {today_date_str}")
    print(f"Log file: {log_file_path}")
    print(f"Total links found matching pattern: {links_found_count}")
    print(f"Links processed attempt: {links_processed}")
    print(f"New archives saved: {new_files_saved}")
    print(f"Updated archives saved: {updated_files_saved}")
    print(f"Diff files saved: {diff_files_saved}")
    print(f"Links skipped/failed: {links_failed}")
    print("-------------------------------\n")


ECMC_archive_main()

## Downloads from FracFocus


In [ ]:
wholeset['job_end_date'] = wholeset.JobEndDate.str.split().str[0]
wholeset['job_end_date'] = pd.to_datetime(wholeset.job_end_date,format="%m/%d/%Y")
wholeset['FF_disc'] = wholeset.apply(lambda x: th.getFFLink(x), axis=1)
wholeset['disc_link'] = wholeset.apply(lambda x: th.getDisclosureLink(APINumber=x.APINumber,
                                                                      disclosureid=x.discID,
                                                                      text_to_show='Open-FF disclosure',
                                                                      use_remote=True,
                                                                      check_if_exists=False), axis=1)
cond = (wholeset.date_changed>=last_repo)&(wholeset.change_type!='removed')
wholeset.disc_link = np.where(cond,' ',wholeset.disc_link)

# correct some occasional errors in TBWV raw input
wholeset['TBWV'] = wholeset.TotalBaseWaterVolume.astype('float64')
wholeset['TBWV'] = wholeset.TBWV.map(lambda x: th.round_sig(x,5))
wholeset['TBWV'] = wholeset.TBWV.fillna(' - ')
# wholeset.columns

In [ ]:
print(f'Number of data set records: {len(wholeset)}')
wholeset.change_type.value_counts()

In [ ]:
wholeset['has more than one'] = np.where(wholeset.APINumber.duplicated(keep=False),'APINumber dupe','')

In [ ]:
# for curated disclosures, get detected flaws

DiDs = wholeset.discID.unique().tolist()
rec_iss = pd.read_parquet(os.path.join(hndl.curr_repo_dir,'record_issues.parquet'),columns=['r_flags','reckey'])
reck = pd.read_parquet(os.path.join(hndl.curr_repo_pkl_dir,'chemrecs.parquet'),columns=['DisclosureId','reckey'])
mg = pd.merge(rec_iss,reck,on='reckey',how='left')
gb = mg[mg.DisclosureId.isin(DiDs)].groupby('DisclosureId',as_index=False)['r_flags'].apply(set)

# fix situation where there are no DiDs from the record_issues df
if len(gb)==0:
    gb = pd.DataFrame(columns=mg.columns)


def str_from_set(x):
    s = ''
    for item in x:
        s += item +' '
    return s

gb['recstr'] = gb.r_flags.map(lambda x: str_from_set(x))

In [ ]:
dis_iss = pd.read_parquet(os.path.join(hndl.curr_repo_dir,'disclosure_issues.parquet'),columns=['d_flags','DisclosureId'])
dis_iss = dis_iss[dis_iss.DisclosureId.isin(DiDs)]
gb = gb.merge(dis_iss,on='DisclosureId',how='outer')
gb = gb.fillna('')
gb['issues'] = gb.d_flags +' '+gb.recstr
wholeset = wholeset.merge(gb[['DisclosureId','issues']],left_on='discID',right_on='DisclosureId',how='left')
wholeset.issues = wholeset.issues.fillna('')

In [ ]:
## watch list summary
## See bottom of page for whole list
# url = 'https://raw.githubusercontent.com/gwallison/FF_issues/master/watch_list.csv'
# wdf = pd.read_csv(url,dtype = {'APINumber':'str'})
url = 'https://raw.githubusercontent.com/gwallison/FF_issues/master/watch_list_master.parquet'
wdf = pd.read_parquet(url)
wdf = wdf.rename({'DisclosureId':'wl_DisclosureId'},axis=1)
# wdf.date_entered = pd.to_datetime(wdf.date_entered,format='%m/%d/%y')
# wdf.FF_report_date = pd.to_datetime(wdf.FF_report_date,format='%m/%d/%y')
# wdf.Blog_date = pd.to_datetime(wdf.Blog_date,format='%m/%d/%y')
# wdf.FF_updates = pd.to_datetime(wdf.FF_updates,format='%m/%d/%y')

apis = wdf.APINumber.unique().tolist()

watchlist_found = pd.merge(wdf,wholeset,on='APINumber',how='inner')
# watchlist_found[['wl_name','change_type']].value_counts()

## Pattern of new disclosure additions
These disclosures are detected as new because their `DisclosureId` number hasn't been in the database before.  Note that it is possible that they are a new version of a previously published disclosure; sometimes operators change disclosures by removing the old one from FracFocus and creating a new one.

In [ ]:
# print(download_dates)
import warnings
warnings.filterwarnings("ignore")
import calplot
calplot.calplot(added_sum,  cmap='Spectral_r', yearlabel_kws={'fontname':'sans-serif'});


- **Blue line** = Cumulative new disclosures added (my include replacements for removed disclosures)
- **Orange line** = New disclosures with detected issues
- **Vertical dashed line** = date of last Open-FF data set generation

In [ ]:
t = wholeset[wholeset.change_type=='added']

gb = t.groupby('date_changed',as_index=False).size()
gb['cs'] = gb['size'].cumsum()
gb = gb[['date_changed','cs']].set_index('date_changed')
ax = gb.cs.plot(title='Number of new disclosures',
               ylabel='cumulative disclosures', xlabel='date changed')

gb = t[t.issues.str.len()>1].groupby('date_changed',as_index=False).size()
gb['with_issues'] = gb['size'].cumsum()
gb = gb[['date_changed','with_issues']].set_index('date_changed')
ax = gb.with_issues.plot(ax=ax)


ax.axvline(last_repo, color="green", linestyle="dashed");


In [ ]:
show_whole = wholeset[wholeset.change_type=='added'].rename({'job_end_date':'job end date','date_changed':'date added',
                            'change_type':'change type'},axis=1)
# show_whole.TBWV = pd.to_numeric(show_whole.TBWV)
try:
    show_whole[['APINumber','FF_disc','disc_link','job end date','StateName','CountyName','OperatorName',
              'TBWV','date added','has more than one','issues']].to_parquet(added_disc_fn)
except:
    print(f'Could not save to {added_disc_fn}')

show_whole[['FF_disc','disc_link','job end date','StateName','CountyName','OperatorName',
          'TBWV','date added','has more than one','issues']].reset_index(drop=True)


### Removed disclosures

In [ ]:
if removed_sum.sum()>0.5:
    calplot.calplot(removed_sum, cmap='Spectral_r', yearlabel_kws={'fontname':'sans-serif'});
else:
    display(md('#### No removed disclosures found'))

In [ ]:
# from: https://www.geeksforgeeks.org/test-the-given-page-is-found-or-not-on-the-server-using-python/#
import requests
def url_ok(url):
    # return True
	# exception block
	try:
		# pass the url into
		# request.head
		response = requests.head(url)
		
		# check the status code
		if response.status_code == 200:
			return True
		else:
			return False
	except requests.ConnectionError as e:
		return e


In [ ]:
def get_other_dfs_param(target_Did,target_api):
    dfs = wholeset[wholeset.APINumber==target_api]
    numdfs = len(dfs)
    otherDids = {}
    for i,row in dfs.iterrows():
        if row.discID != target_Did:
            otherDids[row.discID] = row.date_changed
    return otherDids    

def other_has_different_discID(row,others):
    has_different_dId = False
    for dId in others:
        if dId!=row.discID:
            has_different_dId = True
    return has_different_dId

def other_is_likely_replacement(row,others):
    replacementDid = None
    for dId in others:
        dtime = (row['date removed'] -  others[dId]).days
        if abs(dtime)<31:
            replacementDid = dId
    return replacementDid    

In [ ]:
datelst = get_raw_list()
old_rem = pd.read_parquet(removed_disc_fn)

if removed_sum.sum()>0.5:   #0.5 to account for the 0.001 baseline
    show_whole = wholeset[wholeset.change_type=='removed'].rename({'job_end_date':'job end date',
                                                                   'date_changed':'date removed',
                                                                   'change_type':'change type'},axis=1)
    # ldiffs = []
    # hmto = []
    # for i,row in show_whole.iterrows():
    #     if row['has more than one']:
    #         others = get_other_dfs_param(row.discID,row.APINumber)
    #         # print(f'How many others? {len(others.keys())}')
    #         # Don't deal with well with too many or too few other disclosures
    #         if len(others.keys())!=1:
    #             ldiffs.append(-1)
    #             hmto.append(f'others != 1; instead: {len(others.keys())}')
    #             continue
            
    #         # make sure it is a different discID - otherwise it was added BEFORE removed
    #         if not other_has_different_discID(row,others):
    #             ldiffs.append(-1)
    #             hmto.append('other was added BEFORE this removal')
    #             continue  # ignore other, it was earlier and dropped

    #         # make sure other is a likely REPLACEMENT - the change dates are similar
    #         replacementDid = other_is_likely_replacement(row,others)
    #         if not replacementDid:
    #             ldiffs.append(-1)
    #             hmto.append("other's change date was >30 days different")
    #             continue
                
    #         chg_date = row['date removed'].strftime("%Y-%m-%d")
    #         outfn = get_local_diff_fn(row.APINumber,chg_date)
    #         url = get_diff_url(row.APINumber,chg_date)
    #         if url_ok(url):  # don't remake this diff file if already uploaded
    #             try: 
    #                 # # temp
    #                 # ldiffs.append(0)
    #                 # hmto.append(th.wrap_URL_in_html(url,'compare with new'))
    #                 # continue

    #                 ldiff_df = old_rem[(old_rem.APINumber==row.APINumber) & (old_rem['date removed']==row['date removed'])]['difflib size']
    #                 if len(ldiff_df)>0: # the old df is usable
    #                     ldiffs.append(ldiff_df.iloc[0])
    #                     hmto.append(th.wrap_URL_in_html(url,'compare with new'))
    #                     continue
    #                 else:
    #                     print(' -- regenerating ldiff...') # let it regenerate
    #             except:
    #                 print(f'something went wrong with already generated {row.APINumber}') 

    #         # make the diff file
    #         try:
    #             len_diff = PDF_like_compare(row.APINumber,row.discID,replacementDid,chg_date,datelst,outhtml=outfn)
    #             ldiffs.append(len_diff)
    #             hmto.append(th.wrap_URL_in_html(url,'compare with new'))
    #         except:
    #             ldiffs.append(-1)
    #             hmto.append('error with diff process')
    #     else:
    #         ldiffs.append(-1)
    #         hmto.append('')
    # show_whole['difflib size'] = ldiffs
    # show_whole['has more than one'] = hmto
    show_whole.TBWV = show_whole.TBWV.astype('str')
    # show_whole['difflib size'] = show_whole['difflib size'].astype('str')

    show_whole[['APINumber','FF_disc','disc_link','job end date','StateName','CountyName','OperatorName',
                'TBWV','date removed',
                # 'has more than one','difflib size',
                'issues']].to_parquet(removed_disc_fn)
    
    iShow(show_whole[['FF_disc','disc_link','job end date','StateName','CountyName','OperatorName',
                      'TBWV','date removed',
                      # 'has more than one','difflib size',
                      'issues']].reset_index(drop=True))

### Modified disclosures

In [ ]:
if changed_sum.sum()>0.5:  #0.5 to account for the 0.001 baseline
    calplot.calplot(changed_sum, cmap='Spectral_r', yearlabel_kws={'fontname':'sans-serif'});
else:
    display(md('#### No modified disclosures detected'))

In [ ]:
if changed_sum.sum()>0.5:
    show_whole = wholeset[wholeset.change_type=='modified'].rename({'job_end_date':'job end date','date_changed':'date modified',
                            'change_type':'change type'},axis=1)
    show_whole[['APINumber','FF_disc','disc_link','job end date','StateName','CountyName','OperatorName',
          'TBWV','date modified','has more than one','issues']].to_parquet(modified_disc_fn)
    iShow(show_whole[['FF_disc','disc_link','job end date','StateName','CountyName','OperatorName',
          'TBWV','date modified','has more than one','issues']].reset_index(drop=True))

In [ ]:
# get list of *reported* issues
def add_to_set(s, iset):
    lst = s.split()
    for i in lst:
        iset.add(i)
    return iset

iset = set()
for i, row in wholeset.iterrows():
    iset = add_to_set(row.issues,iset)

### Issues list

In [ ]:
import FF_issues.process_master_files as pmf
pobj = pmf.Process_Master_Files()
df = pobj.process_obj()


#### Discloure-level Issues

In [ ]:
c = df.Flag_id.str[0]=='d'
c1 = df.Flag_id.isin(iset)
t = df[c&c1].copy()
t['flaw_link'] = t.Flag_id.map(lambda x: th.getFlawLink(x))
t[['Title','flaw_link','Warning_level']]

#### Record-level Issues

In [ ]:
c = df.Flag_id.str[0]=='r'
# c1 = df.Flag_id.isin(iset)
t = df[c&c1].copy()
t['flaw_link'] = t.Flag_id.map(lambda x: th.getFlawLink(x))
t[['Title','flaw_link','Warning_level']]

## New CASNumber : IngredientName pairs

In [ ]:
# Are there new casing?
repo_casing = fh.get_casing_df()
changed_casing = pd.DataFrame(casing,columns=['CASNumber','IngredientName'])
mg = pd.merge(changed_casing,repo_casing,on=['CASNumber','IngredientName'], how='left',indicator=True)
mg[mg._merge=='left_only'][['CASNumber','IngredientName']]

In [ ]:
# Are there new Operators?
repo_companies = fh.get_company_df()
complst = repo_companies.rawName.tolist()
newcomp = []
for op in operator:
    if not op in complst:
        newcomp.append(op)
if len(newcomp)> 0:
    display(md('## New Operator names detected'))
    newcomp.sort()
    for item in newcomp:
        display(md(f'##### {item}'))
else:
    display(md('### No new operator names detected'))

___
## Watch list
Wells with changed disclosures that were previously detected with problems

In [ ]:
watchlist_found

## Make daily summary file available
added Sept 2025 - for Christiaan

In [ ]:
rawdir = r"G:\My Drive\production\openFF_archive\raw_dataframes"
dlst = os.listdir(rawdir)
sorted(dlst)
fn = dlst[-1]
t = pd.read_parquet(os.path.join(rawdir,fn))
gb = t.groupby('DisclosureId',as_index=False)[['APINumber','StateName','CountyName','TotalBaseWaterVolume',
                                               'JobStartDate','JobEndDate','Latitude','Longitude']].first()
outfn = os.path.join(r"G:\My Drive\webshare\temp",'daily_raw_summary.parquet')
gb.to_parquet(outfn)
                     
##  save files to google storage
import openFF.common.file_handlers as fh
bucket_name ='open-ff-browser'
blob_name='daily_raw_summary.parquet'
fh.upload_file_to_bucket(bucket_name,blob_name,file_path=outfn,verbose=True,disable_caching=True)